In [13]:
import requests
import json
import pandas as pd
from steam import Steam
from tqdm import tqdm
import csv
import time
import os
from urllib.parse import quote
from bs4 import BeautifulSoup
import html_to_json

In [2]:
# Per recuperare le recensioni degli utenti steam procedi nella seguente maniera:
# 1- Prendi le prime 100 recensioni di ogni gioco che ha recensioni
# 2- Per ognuna di queste prendi lo steam id del recensore
# 3- A questo punto interroga il get player summaries per ogni steam id recuperato

# Potrebbe essere una buona idea parsare le recensioni per recuperare prima
# la lista dei giocatori e poi interrogare direttamente il get player summaries

In [3]:
KEY = '0BAEAA44F5C3C9967C53E41AE9B50F25'

In [4]:
# Come prima cosa recupera 100 reviews per un gioco sample

app_id = 2231450
steam_id = 76561198151018729

In [8]:
# url API a cui effettuare la richiesta per le reviews

def get_game_reviews(app_id):
    steam_user_ids = []
    url = f'http://store.steampowered.com/appreviews/{app_id}'
    cursor = '*'
    old_cursor = ''

    while cursor != old_cursor:
        old_cursor = cursor
        encoded_cursor = quote(cursor)
        print(cursor)

        params = {
            'json': 1,
            'language': 'all',
            'cursor': encoded_cursor,
            'purchase_type': 'all',
        }
    
        response = requests.get(url, params=params)
        print(response.status_code)

        if response.status_code == 200:
            data = response.json()
            if data['success'] == 1:
                reviews = data['reviews']
                cursor = data['cursor']
                for review in reviews:
                    steam_user_ids.append(review['author']['steamid'])
                print(len(steam_user_ids))
            else:
                print('Error: Unable to retrieve game reviews.')
        
    return steam_user_ids

In [9]:
data = get_game_reviews(app_id)

*
200
20
AoIIP1sqBnzC+owE
200
40


In [7]:
len(set(data))

20

In [14]:
# https://steamcommunity.com/profiles/76561199227008553/recommended/
steam_id = 76561199227008553
url = f'https://steamcommunity.com/profiles/{steam_id}/recommended/'
response = requests.get(url)
html_string = response.text
output_json = html_to_json.convert(html_string)
#data = response.json()
#data

In [16]:
with open('output_files/reviews/prova.json', 'w') as F:
    json.dump(output_json, F)

In [22]:
import steamreviews

app_id = 2141700
review_dict, query_count = steamreviews.download_reviews_for_app_id(app_id)

[appID = 2141700] expected #reviews = 257


In [23]:
query_count

3